In [1]:
import os 
os.chdir('../src/')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
from pandas.io.json import json_normalize
from IPython.display import display

from d00_utils.load_confs import load_credentials, load_paths
from d00_utils.data_loader_sql import DataLoaderSQL
from d01_data_processing.data_cleaning import *
from d01_data_processing.spotify_user import SpotifyUser

creds = load_credentials()
paths = load_paths()

Current working directory is now:  C:\Users\Caroline Wang\OneDrive\Duke\Senior Year\CS316\Project\src


In this notebook we will create functions to pull data from the VM database and process the data into features 

## Query Data From VM Database Using Python

1. Need a SQL query that returns the top tracks for a FIXED listener_id, time_span
For these top tracks, need a SQL query that returns the release date of each track (associated with the album of the track), popularity of each track, audio features of each track (i.e., energy level, danceability, speechiness, instrumentalness, valence)

Get toptracks info from tracks table. Still need to get toptracks genre from the AlbumHasGenre table. 
```
 select track_pop, acousticness, danceability, energy, valence, 
        loudness, tempo, instrumentalness, speechiness, 
        mode, time_signature, liveness
        from tracks as t1
        where t1.track_id in (
                     select t2.track_id 
                      from toptracks as t2 
                      where listener_id='{listener_id}'and time_span='long_term')                  
```


2. Need a SQL query that returns the top artists for a FIXED listener_id, time_span
For these top artists, need a SQL query that returns the genres of each artist, the popularity of each artist, number followers of each artist

Get topartists info from tracks table. Still need to get topartists genre info from the ArtistHasGenre table 
```
select * 
from artisthasgenre 
natural join 
(select a1.artist_id, a1.artist_pop, a1.num_followers 
from artists as a1 
where a1.artist_id in 
(select a2.artist_id 
 from topartists as a2
 where listener_id='ninjakichi' and time_span='long_term' ))
as artist_info
```

In [2]:
dl = DataLoaderSQL(creds=creds,
                   paths=paths)

listener_id = "ninjakichi"
num_tracks = 20


## Queried Data Into Features 

#### Track related features 

1. Avg energy level, avg danceability, avg speechiness, avg instrumentalness, avg valence of your top tracks from x-term songs, weighted by track_pop, a number from 0 - 100 (100 being most popular)
       - Energy level, acousticness, danceability, speechiness, instrumentalness, liveness, loudness, speechiness, valence, are from 0 -1
2. Tempo: maxed out at around 220 bpm
3. Mode: 1 is major, 0 is minor 
       - Percent of songs that are major

In [39]:

track_cmd = f"""
        select track_pop, acousticness, danceability, energy, valence, 
        loudness, tempo, instrumentalness, speechiness, 
        mode, time_signature, liveness
        from tracks as t1
        where t1.track_id in (
                     select t2.track_id 
                      from toptracks as t2 
                      where listener_id='{listener_id}'and time_span='long_term')                  
            """

track_info = dl.select_from_table(sql=track_cmd)
cols = track_info.columns
track_info[cols] = track_info[cols].apply(pd.to_numeric, errors='coerce')
track_info.head()

track_pop  acousticness  danceability  energy  valence  loudness    tempo  \
0         23        0.6750         0.197   0.320    0.108   -16.696   83.893   
1         73        0.2280         0.653   0.816    0.816    -4.353  178.086   
2         83        0.2570         0.671   0.373    0.732   -18.064   92.717   
3         73        0.0901         0.446   0.952    0.624    -5.321  125.303   
4         74        0.6490         0.571   0.307    0.108   -10.958   83.720   

   instrumentalness  speechiness  mode  time_signature  liveness  
0          0.748000       0.0387     1               4    0.8030  
1          0.000000       0.1670     1               4    0.0967  
2          0.000079       0.0323     1               4    0.0481  
3          0.085700       0.0523     1               4    0.1120  
4          0.000000       0.0499     0               4    0.1780

In [166]:
# compute average weighted by the idf of track_pop 
def compute_inv_pop(series):
    '''vec is a pd series of integers from 0 to 100'''
    return np.log(100/series).replace({np.inf: 5.7})
    
def compute_weighted_avg(series, weights):
    '''compute the weighted avg of series'''
    return np.average(series, weights=weights)
    

def compute_track_features(track_info:pd.DataFrame,
                           num_tracks=20):
    """
    computes the features for an individual 
    """
    # top tracks track features 
    # avg weighted by the track inverse popularity 
    col_list= ['acousticness', 'danceability', 'energy', 
                'valence', 'loudness', 'tempo', 'instrumentalness',
                'speechiness', 'time_signature', 'liveness'] 
    track_info['inv_pop'] = compute_inv_pop(track_info['track_pop'])
    features_tracks_dict = {f"avg_{col}": compute_weighted_avg(series=track_info[col],
                                                               weights=track_info['inv_pop']) 
                     for col in col_list }
    # features_dict
    features_tracks_dict["%_major"] = np.sum(track_info['mode'])/num_tracks
    return features_tracks_dict

In [167]:
compute_track_features(track_info)

{'avg_acousticness': 0.3849086253539318,
 'avg_danceability': 0.49208783113630666,
 'avg_energy': 0.5548612411256385,
 'avg_valence': 0.45680829315132543,
 'avg_loudness': -11.53396334195145,
 'avg_tempo': 112.33350984716522,
 'avg_instrumentalness': 0.6190045290597034,
 'avg_speechiness': 0.0626400337980007,
 'avg_time_signature': 3.803718044328438,
 'avg_liveness': 0.167641725741742,
 '%_major': 0.65}

#### Album related features 

1. % of artists you listen to that fall into well-defined subcategories 
2. Avg popularity of your top artists (artist_pop)


In [151]:
artist_cmd = f"""
            select * 
            from artisthasgenre 
            natural join 
                (select a1.artist_id, a1.artist_pop, a1.num_followers 
                 from artists as a1 
                 where a1.artist_id in 
                    (select a2.artist_id 
                     from topartists as a2
                     where listener_id='ninjakichi' and time_span='long_term' ))
                 as artist_info
            """
artist_info = dl.select_from_table(sql=artist_cmd)
artist_info.head()

artist_id         genre_name  artist_pop  num_followers
0  1HY2Jd0NmPuamShAr6KMms          dance pop          87       11076948
1  1HY2Jd0NmPuamShAr6KMms                pop          87       11076948
2  1HY2Jd0NmPuamShAr6KMms      post-teen pop          87       11076948
3  3dRfiJ2650SZu6GbydcHNb         soundtrack          75         849382
4  1hCkSJcXREhrodeIHQdav8  german soundtrack          74         396012

In [168]:
genre_map = {
                # new
                'alternative': 'alternative', 
                'indie': 'alternative', 
                'experimental': 'experimental', 
                'avant': 'experimental', 
                
                # typical gentres 
                'country': 'country', 
                'folk': 'country', 
                'rock': 'rock', 
                'punk': 'rock', 
                'metal': 'metal',
                'rap': 'hip_hop', 
                'hip': 'hip_hop', 
                'hop': 'hip_hop',
                'trap': 'hip_hop', 
                'pop': 'pop', 
                
                # r&b, african-american insp. 
                'r&b': 'r&b_soul', 
                'soul': 'r&b_soul', 
                'funk': 'r&b_soul',
                'afro': 'jazz', 
                'jazz': 'jazz',
                
                # religious 
                'gospel': 'christian', 
                'christian': 'christian', 

                # ethnic
                'reggae': 'latin', 
                'latin': 'latin', 
                 
                # soundtrack 
                'soundtrack': 'soundtrack',
                'video': 'soundtrack', 
                'score': 'soundtrack',
                'tunes': 'soundtrack', 
                'cartoon': 'soundtrack', 
                'anime': 'soundtrack', 
                'otacore': 'soundtrack',
                
                # classical type / vocal
                'classical': 'classical', 
                'orchestra': 'classical',                
                'chamber': 'classical', 
                'contemporary': 'contemporary', 
                # vocal
                'capella': 'vocal',
                'choir': 'vocal',
                'alto': 'vocal',
                'soprano': 'vocal', 
                'vocal': 'vocal',
                
                # electronic 
                'edm': 'edm', 
                'rave': 'edm', 
                'house': 'edm', 
                'tech': 'edm',
                'room': 'edm',
                'step': 'edm', 
                'dance': 'edm', 
                'electronic': 'edm',
                'trance': 'edm',
                
                # misc chill : 
                'lo-fi': 'chill',
                'meditation': 'chill', 
                'drone': 'chill', 
                'focus': 'chill', 
                'zen': 'chill'
               }

def invert_dictionary(adict:dict):
    '''keys->values, values->keys'''
    inv_map = {}
    for k, v in adict.items():
        inv_map[v] = inv_map.get(v, [])
        inv_map[v].append(k)
    return inv_map

In [169]:
def compute_artists_features(artist_info:pd.DataFrame, 
                             genre_map:dict, 
                             num_tracks=20):
    '''compute the top artists features for an individual'''

    genre_str_arr = np.array(artist_info['genre_name'], dtype=str)
    inv_genre_map = invert_dictionary(genre_map)
    
    for genre, subgenres in inv_genre_map.items(): 
        # test for subgenre in each entry of genre_str_arr
        subgenre_bool_filters = []
        for subgen in subgenres: 
            subgenre_bool_filters.append((np.core.defchararray.find(genre_str_arr, 
                                                                    subgen) != -1))
        # elementwise logical or on all subgenre boolean arrays 
        logical_or_bool = np.logical_or.reduce(subgenre_bool_filters)
        artist_info[f"is_{genre}"] = logical_or_bool

    artist_genres = artist_info.copy()
    del artist_genres['genre_name']
    artist_genres = artist_genres.groupby(['artist_id', 'artist_pop', 'num_followers']).any()

    features_artists_dict = (artist_genres.reset_index().drop(['artist_id', 'artist_pop', 'num_followers'], axis=1).sum()/num_tracks).to_dict()
    # compute avg artist popularity and rescale
    features_artists_dict['avg_artist_pop'] = np.mean(artist_info['artist_pop'])/100

    return features_artists_dict

In [170]:
compute_artists_features(artist_info=artist_info, 
                         genre_map=genre_map, 
                         num_tracks=num_tracks)

{'is_alternative': 0.1,
 'is_experimental': 0.0,
 'is_country': 0.0,
 'is_rock': 0.05,
 'is_metal': 0.05,
 'is_hip_hop': 0.0,
 'is_pop': 0.25,
 'is_r&b_soul': 0.0,
 'is_jazz': 0.0,
 'is_christian': 0.0,
 'is_latin': 0.0,
 'is_soundtrack': 0.6,
 'is_classical': 0.2,
 'is_contemporary': 0.0,
 'is_vocal': 0.0,
 'is_edm': 0.05,
 'is_chill': 0.0,
 'avg_artist_pop': 0.5884615384615385}

In [172]:
features_dict = {**features_artists_dict, **features_tracks_dict}

### Compute features for all individuals

In [173]:
# dl.select_from_table(sql='select * from listeners')

listener_id   display_name       username  \
0     zan6n9pb4njsd6dm20zd5ptt4         Martha     martha1234   
1                    1214600613     1214600613  selen_berkman   
2                    ninjakichi     ninjakichi    sophie_wang   
3  314xcqarki42gnkosjtfplluumya  Caroline Wang   caroline1717   
4                elise_brown212    Elise Brown     EliseBrown   
5                     bnativi17      bnativi17      bnativi17   

                                        access_token  expires_in  \
0  BQD7lQaovdkPFq7vDAa8bq4HiiMg9uEobdvyCgXO4Ps8Z6...        3600   
1  BQAUnE3tYDiX2MBRJmNeiSTP6aaOYr3U3F4OudeSaU-zlR...        3600   
2  BQANd8OPet8aY0MFKnix5TMD50tJ7Rvb12yWDmPA0Elu3D...        3600   
3  BQC9lXtZxO9sR-ZgdQAYTjgYqfSo2xELLsLL89CrhOhKPW...        3600   
4  BQAspAB1m_W0fBztMmEEYnoQVXEBx3El2t2M8qWwP78xPy...        3600   
5  BQD6a8c8VPsW0krT_x-P21XEOCpxiJ9veUOT4CEdc_O-EI...        3600   

                                       refresh_token  \
0  AQBb_0F6xYTe1p7txMsDctvbppQmZBcx-90cUP1bWw4CE2...   
1  AQAeWzbD9SSMnArodMedzdz_vdqQ_u9hTb6tjuS2Se0QbK...   
2  AQBt7scAWmbla_qfZQGydS_UV3C_RhALKvyS3YWJbF0BLW...   
3  AQA7mvkHhhEdRDz-Xdsgl3x8uyJiX1ZhZjt5hgdHKLmYPT...   
4  AQBLM-O2khW85lfRHSw7HHSypdHvbiK0NMKCLi6QiE9Ueh...   
5  AQB-VkBhZ_sr6xw7gXZwpzrdD5X5uGzEbkbSFCV9j5XQ4L...   

                                               scope  num_followers  \
0  user-library-read user-follow-read user-read-r...              2   
1  user-library-read user-follow-read user-read-r...             51   
2  user-library-read user-follow-read user-read-r...              0   
3  user-library-read user-follow-read user-read-r...              2   
4  user-library-read user-follow-read user-read-r...             11   
5  user-library-read user-follow-read user-read-r...             10   

                                  listener_image_url  
0                                               NULL  
1                                               NULL  
2                                               NULL  
3  https://platform-lookaside.fbsbx.com/platform/...  
4  https://profile-images.scdn.co/images/userprof...  
5                                               NULL

In [183]:
# query tracks and artist data for all individuals 

listener_ids = dl.select_from_table(sql='select listener_id from listeners')

top_track_info, top_artist_info = [], []
for listener_id in np.array(listener_ids):
    listener_id = listener_id[0]
    
    # top track info 
    track_cmd = f"""
                select track_pop, acousticness, danceability, energy, valence, 
                loudness, tempo, instrumentalness, speechiness, 
                mode, time_signature, liveness
                from tracks as t1
                where t1.track_id in (
                  select t2.track_id 
                  from toptracks as t2 
                  where listener_id = '{listener_id}'
                  and time_span='long_term')                  
                """

    track_df = dl.select_from_table(sql=track_cmd)
    track_df['listener_id'] = listener_id
    top_track_info.append(track_df)
    
    # top artist info 
    artist_cmd = f"""
            select * 
            from artisthasgenre 
            natural join 
                (select a1.artist_id, a1.artist_pop, a1.num_followers 
                 from artists as a1 
            where a1.artist_id in 
              (select a2.artist_id 
               from topartists as a2
               where listener_id = '{listener_id}' 
               and time_span='long_term' ))
            as artist_info
            """
    artist_df = dl.select_from_table(sql=artist_cmd)
    artist_df['listener_id'] = listener_id
    top_artist_info.append(artist_df)
    
top_track_info = pd.concat(top_track_info, axis=0)
cols = top_track_info.columns[:-1]
top_track_info[cols] = top_track_info[cols].apply(pd.to_numeric, errors='coerce')

top_artist_info = pd.concat(top_artist_info, axis=0)

In [184]:
display(top_track_info.head())
display(top_artist_info.head())

track_pop  acousticness  danceability  energy  valence  loudness    tempo  \
0         58       0.30600         0.553   0.314    0.585    -9.217   91.700   
1         64       0.00228         0.953   0.723    0.221    -6.437   99.969   
2         62       0.10100         0.863   0.625    0.456    -6.323   83.980   
3         61       0.76800         0.453   0.362    0.181   -11.154  134.949   
4         57       0.94300         0.606   0.264    0.319   -10.152   95.021   

   instrumentalness  speechiness  mode  time_signature  liveness  \
0          0.000578       0.0396     1               4    0.1040   
1          0.000000       0.2480     1               4    0.3360   
2          0.000004       0.3130     1               4    0.1660   
3          0.014800       0.0319     1               4    0.0998   
4          0.000056       0.0243     1               4    0.1180   

                 listener_id  
0  zan6n9pb4njsd6dm20zd5ptt4  
1  zan6n9pb4njsd6dm20zd5ptt4  
2  zan6n9pb4njsd6dm20zd5ptt4  
3  zan6n9pb4njsd6dm20zd5ptt4  
4  zan6n9pb4njsd6dm20zd5ptt4

artist_id                   genre_name  artist_pop  \
0  3vDpQbGnzRbRVirXlfQagB        alternative americana          65   
1  3vDpQbGnzRbRVirXlfQagB  nashville singer-songwriter          65   
2  3vDpQbGnzRbRVirXlfQagB                new americana          65   
3  3vDpQbGnzRbRVirXlfQagB               outlaw country          65   
4  3vDpQbGnzRbRVirXlfQagB                   roots rock          65   

   num_followers                listener_id  
0         371086  zan6n9pb4njsd6dm20zd5ptt4  
1         371086  zan6n9pb4njsd6dm20zd5ptt4  
2         371086  zan6n9pb4njsd6dm20zd5ptt4  
3         371086  zan6n9pb4njsd6dm20zd5ptt4  
4         371086  zan6n9pb4njsd6dm20zd5ptt4

In [187]:
features_df = []
for listener_id in np.array(listener_ids):
    listener_id = listener_id[0]
    track_listener_info = top_track_info[top_track_info["listener_id"] == listener_id]
    artist_listener_info = top_artist_info[top_artist_info["listener_id"] == listener_id]
    
    track_feat = compute_track_features(track_listener_info)
    artist_feat = compute_artists_features(artist_listener_info, 
                                           genre_map)
    
    features = {**track_feat, 
                **artist_feat,
                "listener_id": listener_id}
    features_df.append(features)

features_df = pd.DataFrame(features_df)
features_df

C:\Users\Caroline Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Caroline Wang\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


avg_acousticness  avg_danceability  avg_energy  avg_valence  avg_loudness  \
0          0.524948          0.632793    0.496907     0.420923     -9.222899   
1          0.063211          0.658945    0.781255     0.460777     -4.950297   
2          0.384909          0.492088    0.554861     0.456808    -11.533963   
3          0.481755          0.428288    0.449242     0.310486    -11.866537   
4          0.463738          0.513752    0.543231     0.426426    -10.188628   
5          0.024226          0.583550    0.889773     0.288178     -4.829371   

    avg_tempo  avg_instrumentalness  avg_speechiness  avg_time_signature  \
0  101.786011              0.044895         0.160565            3.965484   
1  123.898813              0.009605         0.063384            3.971549   
2  112.333510              0.619005         0.062640            3.803718   
3  105.152720              0.282600         0.070260            3.607879   
4  112.664519              0.002306         0.170076            3.940402   
5  139.831849              0.285106         0.096484            4.000000   

   avg_liveness  ...  is_jazz is_christian  is_latin  is_soundtrack  \
0      0.183360  ...      0.1         0.00      0.00           0.00   
1      0.214984  ...      0.0         0.00      0.05           0.00   
2      0.167642  ...      0.0         0.00      0.00           0.60   
3      0.172975  ...      0.0         0.00      0.00           0.00   
4      0.250314  ...      0.0         0.25      0.00           0.05   
5      0.359377  ...      0.0         0.00      0.00           0.00   

   is_classical  is_contemporary  is_vocal  is_edm  is_chill  avg_artist_pop  
0          0.00             0.20       0.0    0.50       0.0        0.765400  
1          0.00             0.05       0.0    0.65       0.0        0.800595  
2          0.20             0.00       0.0    0.05       0.0        0.588462  
3          0.25             0.05       0.0    0.30       0.0        0.705000  
4          0.00             0.05       0.0    0.25       0.0        0.741905  
5          0.00             0.00       0.0    0.65       0.0        0.686832  

[6 rows x 30 columns]